# Open Tensorboard

In [2]:
#%load_ext tensorboard
#%tensorboard --logdir lightning_logs  --samples_per_plugin  images=100

In [3]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from matplotlib.ticker import FormatStrFormatter
from scipy.signal import savgol_filter    
from operator import itemgetter

In [4]:
def plot_mean_std_max(log_dir, y_keys=["valid_f1", "valid_accurancy"], rng=[30, 75], k=0):
    for key in y_keys:
        tmp, tmp2, tmp3, tmp4 = [], [], [], []
        print("------", key, "------")
        for itm in log_dir:
            event_accumulator = EventAccumulator(itm)
            event_accumulator.Reload()
            y = np.array([x.value for x in event_accumulator.Scalars(key)])

            tmp.append(np.max(y[rng[0]:rng[1]]))
            tmp2.append(y[rng[0]:rng[1]])
            tmp3.append(y[rng[1]])
            tmp4.append([itm.split("\\")[1], round(y[rng[1]], 5), round(y[rng[0]:rng[1]].mean(), 5), round(np.std(y[rng[0]:rng[1]]), 5), round(np.max(y[rng[0]:rng[1]]), 5), round(np.min(y[rng[0]:rng[1]]), 5)])
        
        for i in sorted(tmp4, key=itemgetter(k), reverse=True): print('{:<40} last={:<10} mean={:<10} std={:<10} max={:<10} min={:<10}'.format(*i))
        print()
        print("mean of max in", rng, 'mean={:<10} std={:<10}'.format(np.mean(tmp), np.std(tmp)))
        print("mean of all", rng, 'mean={:<10} std={:<10}'.format(np.mean(tmp2), np.std(tmp2)))
        print("mean of last in", rng[1], 'mean={:<10} std={:<10}'.format(np.mean(tmp3), np.std(tmp3)))

In [5]:
def plot_multiple_graph_vert(log_dir, y_keys, range, av, dontplot):   
    plot_mean_std_max(log_dir, y_keys, av)
    if dontplot: return

    fig, axs = plt.subplots(1,1, sharex=True, figsize=(20, 8))
    try:
        if len(axs): pass
    except:
        axs = [axs]
        
    for ax, key in zip(axs, y_keys):
        ax.yaxis.set_major_locator(plt.MaxNLocator(20))
        ax.xaxis.set_major_locator(plt.MaxNLocator(20))
        ax.yaxis.set_major_formatter(FormatStrFormatter('%0.2f'))
        ax.yaxis.set_tick_params(labelbottom=True)
        for itm in log_dir:
            event_accumulator = EventAccumulator(itm)
            event_accumulator.Reload()
            
            x = np.unique([x.value for x in event_accumulator.Scalars("epoch")])
            y = [x.value for x in event_accumulator.Scalars(key)]
            df = pd.DataFrame({"epoch": x[range[0]:range[1]], key: y[range[0]:range[1]]})
            
            sns.lineplot(data=df, x="epoch", y=key, ax=ax, legend='brief', label=itm.split("\\")[1])
            ax.get_legend().remove()

    handles, labels = plt.gca().get_legend_handles_labels()
    fig.legend(handles, labels, loc='right', bbox_to_anchor=(0.1,1))

def plotter_vert(subfolders_to_plot, range, av, dontplot=False, which=["valid_f1"]):
    #plot_multiple_graph_vert(subfolders_to_plot, ["train_loss", "train_f1", "train_accurancy"], range, av, dontplot=dontplot)
    #plot_multiple_graph_vert(subfolders_to_plot, ["valid_loss", "valid_f1", "valid_accurancy"], range, av, dontplot=dontplot)
    plot_multiple_graph_vert(subfolders_to_plot, which, range, av, dontplot=dontplot)

In [6]:
subfolders = [ f.path for f in os.scandir("lightning_logs") if f.is_dir() ]

def filtering_subfolders(itm_to_print, subf=None):
    subf = subf if subf else subfolders
    filter = np.logical_or.reduce([[i in itm  for itm in subf] for i in itm_to_print])
    return [i for i,j in zip(subf, filter) if j == True]

def filtering_sub_stack(itm_to_print):
        subf = subfolders
        for i in itm_to_print:
            filter = np.logical_or.reduce([[i in itm  for itm in subf]])
            subf = [i for i,j in zip(subf, filter) if j == True]
        return subf

#filtering_subfolders(["v3"], filtering_subfolders(["seed-42"]))
#filtering_sub_stack(["seed-42", "v3"])

In [7]:
#subfolders

## Baseline

In [8]:
plotter_vert(filtering_sub_stack(["bert_512_perm-no", "3.000e-6"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_3.000e-6_seed-42_rr     last=0.60513    mean=0.6076     std=0.0355     max=0.67969    min=0.54945   
bert_512_perm-no_3.000e-6_seed-42        last=0.55385    mean=0.55804    std=0.04204    max=0.64341    min=0.48864   
bert_512_perm-no_3.000e-6_seed-1M        last=0.65217    mean=0.63185    std=0.03029    max=0.65823    min=0.51977   
bert_512_perm-no_3.000e-6_seed-1998      last=0.62281    mean=0.61971    std=0.01963    max=0.6556     min=0.56701   
bert_512_perm-no_3.000e-6_seed-0         last=0.63717    mean=0.6266     std=0.02177    max=0.66667    min=0.58095   

mean of max in [30, 50] mean=0.6607189178466797 std=0.012059381086502572
mean of all [30, 50] mean=0.6087591218948364 std=0.040863715946530384
mean of last in 50 mean=0.6142247080802917 std=0.03396620392313449


In [9]:
plotter_vert(filtering_sub_stack(["bert_512_perm-no", "4.000e-6"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_4.000e-6_seed-42_rr     last=0.61463    mean=0.61463    std=0.01754    max=0.6381     min=0.57754   
bert_512_perm-no_4.000e-6_seed-42        last=0.61404    mean=0.59246    std=0.03276    max=0.66412    min=0.49162   
bert_512_perm-no_4.000e-6_seed-1M        last=0.61538    mean=0.64624    std=0.01513    max=0.66667    min=0.60204   
bert_512_perm-no_4.000e-6_seed-1998      last=0.64889    mean=0.63167    std=0.01952    max=0.66667    min=0.57923   
bert_512_perm-no_4.000e-6_seed-0         last=0.58937    mean=0.59982    std=0.0228     max=0.64602    min=0.55497   

mean of max in [30, 50] mean=0.656313693523407 std=0.01194370598485594
mean of all [30, 50] mean=0.6169641083478927 std=0.029947810795511075
mean of last in 50 mean=0.6164629459381104 std=0.01895132641843224


In [10]:
plotter_vert(filtering_sub_stack(["bert_512_perm-no", "5.000e-6"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_5.000e-6_seed-42_rr     last=0.58       mean=0.61104    std=0.02279    max=0.65094    min=0.55135   
bert_512_perm-no_5.000e-6_seed-42        last=0.64035    mean=0.62507    std=0.02189    max=0.65823    min=0.57711   
bert_512_perm-no_5.000e-6_seed-1M        last=0.65741    mean=0.64355    std=0.02134    max=0.67234    min=0.60287   
bert_512_perm-no_5.000e-6_seed-1998      last=0.63793    mean=0.59958    std=0.02075    max=0.6383     min=0.55238   
bert_512_perm-no_5.000e-6_seed-0         last=0.63551    mean=0.64804    std=0.01166    max=0.6696     min=0.62326   

mean of max in [30, 50] mean=0.6578826189041138 std=0.012475925217823992
mean of all [30, 50] mean=0.6254568064212799 std=0.02733306435144415
mean of last in 50 mean=0.6302406668663025 std=0.02627289267744531


In [11]:
plotter_vert(filtering_sub_stack(["bert_512_perm-no", "6.000e-6"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_6.000e-6_seed-42_rr     last=0.63594    mean=0.59477    std=0.02123    max=0.63317    min=0.54839   
bert_512_perm-no_6.000e-6_seed-42        last=0.65403    mean=0.6325     std=0.02045    max=0.67593    min=0.6       
bert_512_perm-no_6.000e-6_seed-1M        last=0.64655    mean=0.63428    std=0.01637    max=0.66957    min=0.60784   
bert_512_perm-no_6.000e-6_seed-1998      last=0.60185    mean=0.5981     std=0.02776    max=0.64602    min=0.54737   
bert_512_perm-no_6.000e-6_seed-0         last=0.66055    mean=0.65128    std=0.00974    max=0.66986    min=0.63303   

mean of max in [30, 50] mean=0.6589062213897705 std=0.016443078643613298
mean of all [30, 50] mean=0.6221874636411667 std=0.029777886168485368
mean of last in 50 mean=0.6397854447364807 std=0.020656900676106104


In [12]:
plotter_vert(filtering_sub_stack(["bert_512_perm-no", "7.000e-6"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_7.000e-6_seed-42_rr     last=0.58128    mean=0.57884    std=0.02518    max=0.62338    min=0.5291    
bert_512_perm-no_7.000e-6_seed-42        last=0.56311    mean=0.57822    std=0.02494    max=0.63478    min=0.5285    
bert_512_perm-no_7.000e-6_seed-1M        last=0.65487    mean=0.66688    std=0.01449    max=0.69198    min=0.64039   
bert_512_perm-no_7.000e-6_seed-1998      last=0.65741    mean=0.62985    std=0.0188     max=0.66038    min=0.58883   
bert_512_perm-no_7.000e-6_seed-0         last=0.64789    mean=0.6436     std=0.01571    max=0.6696     min=0.61751   

mean of max in [30, 50] mean=0.6560246467590332 std=0.024549791812874616
mean of all [30, 50] mean=0.6194781976938247 std=0.040881224617793546
mean of last in 50 mean=0.6209099054336548 std=0.040310346584489086


## Permutation

In [13]:
plotter_vert(filtering_subfolders(["bert_512_perm-no_5.000e-6_seed",]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_5.000e-6_seed-42_rr     last=0.58       mean=0.61104    std=0.02279    max=0.65094    min=0.55135   
bert_512_perm-no_5.000e-6_seed-42        last=0.64035    mean=0.62507    std=0.02189    max=0.65823    min=0.57711   
bert_512_perm-no_5.000e-6_seed-1M        last=0.65741    mean=0.64355    std=0.02134    max=0.67234    min=0.60287   
bert_512_perm-no_5.000e-6_seed-1998      last=0.63793    mean=0.59958    std=0.02075    max=0.6383     min=0.55238   
bert_512_perm-no_5.000e-6_seed-0         last=0.63551    mean=0.64804    std=0.01166    max=0.6696     min=0.62326   

mean of max in [30, 50] mean=0.6578826189041138 std=0.012475925217823992
mean of all [30, 50] mean=0.6254568064212799 std=0.02733306435144415
mean of last in 50 mean=0.6302406668663025 std=0.02627289267744531


In [14]:
plotter_vert(filtering_subfolders(["bert_512_perm-yes_5.000e-6_seed",]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
bert_512_perm-yes_5.000e-6_seed-42_rr    last=0.66667    mean=0.6301     std=0.03651    max=0.67299    min=0.52761   
bert_512_perm-yes_5.000e-6_seed-42       last=0.65198    mean=0.64498    std=0.0128     max=0.67544    min=0.62037   
bert_512_perm-yes_5.000e-6_seed-1M       last=0.64253    mean=0.64001    std=0.0159     max=0.66667    min=0.6       
bert_512_perm-yes_5.000e-6_seed-1998     last=0.62037    mean=0.60368    std=0.01936    max=0.63436    min=0.55769   
bert_512_perm-yes_5.000e-6_seed-0        last=0.63348    mean=0.62019    std=0.01605    max=0.64889    min=0.57971   

mean of max in [30, 50] mean=0.659668231010437 std=0.015695091796069476
mean of all [30, 50] mean=0.6277931970357895 std=0.02635003971504039
mean of last in 50 mean=0.6430075168609619 std=0.015771252072803787


## Adapter

In [15]:
plotter_vert(filtering_subfolders(["bert_512_perm-no_5.000e-6_seed",]), [0, 60], [30, 60], dontplot=True)

------ valid_f1 ------
bert_512_perm-no_5.000e-6_seed-42_rr     last=0.62069    mean=0.60762    std=0.02233    max=0.65094    min=0.55135   
bert_512_perm-no_5.000e-6_seed-42        last=0.63636    mean=0.62863    std=0.01929    max=0.65823    min=0.57711   
bert_512_perm-no_5.000e-6_seed-1M        last=0.68103    mean=0.64886    std=0.02006    max=0.67544    min=0.60287   
bert_512_perm-no_5.000e-6_seed-1998      last=0.60094    mean=0.60833    std=0.02219    max=0.64317    min=0.55238   
bert_512_perm-no_5.000e-6_seed-0         last=0.63063    mean=0.64434    std=0.01323    max=0.6696     min=0.61395   

mean of max in [30, 60] mean=0.6594770312309265 std=0.011811531105101004
mean of all [30, 60] mean=0.6275565795103709 std=0.026245699205816833
mean of last in 60 mean=0.6339314699172973 std=0.02645508634769073


In [16]:
plotter_vert(filtering_sub_stack(["bert-base_adapter"]), [0, 60], [30, 60], dontplot=True)

------ valid_f1 ------
bert-base_adapter_512_42                 last=0.58706    mean=0.59922    std=0.03051    max=0.65728    min=0.53552   
bert-base_adapter_512_1M                 last=0.62559    mean=0.60867    std=0.03696    max=0.66116    min=0.50633   
bert-base_adapter_512_1998               last=0.60204    mean=0.63072    std=0.02077    max=0.68519    min=0.58291   
bert-base_adapter_512_0                  last=0.61883    mean=0.56323    std=0.08036    max=0.64545    min=0.30534   

mean of max in [30, 60] mean=0.6622684299945831 std=0.014439833667445317
mean of all [30, 60] mean=0.6004602747658888 std=0.0537537508352139
mean of last in 60 mean=0.6083829998970032 std=0.015000688532568626


In [17]:
plotter_vert(filtering_sub_stack(["sup-simcse-bert_seed"]), [0, 50], [30, 50], dontplot=True)

------ valid_f1 ------
sup-simcse-bert_seed-42                  last=0.66667    mean=0.65847    std=0.02057    max=0.69058    min=0.60104   
sup-simcse-bert_seed-1M                  last=0.63959    mean=0.62754    std=0.01942    max=0.65138    min=0.59296   
sup-simcse-bert_seed-1998                last=0.69565    mean=0.65756    std=0.02571    max=0.70386    min=0.60317   
sup-simcse-bert_seed-0                   last=0.59783    mean=0.62561    std=0.01145    max=0.64186    min=0.59375   

mean of max in [30, 50] mean=0.6719205528497696 std=0.025953150484813794
mean of all [30, 50] mean=0.6422946602106094 std=0.025412283423985126
mean of last in 50 mean=0.649934709072113 std=0.0360287736515787


In [18]:
plotter_vert(filtering_sub_stack(["sup-simcse-bert_adapter"]), [0, 60], [30, 60], dontplot=True)

------ valid_f1 ------
sup-simcse-bert_adapter_512_42           last=0.67308    mean=0.65823    std=0.01788    max=0.7027     min=0.60825   
sup-simcse-bert_adapter_512_1M           last=0.65403    mean=0.63867    std=0.03352    max=0.70423    min=0.56684   
sup-simcse-bert_adapter_512_1998         last=0.61538    mean=0.64805    std=0.02355    max=0.69264    min=0.58378   
sup-simcse-bert_adapter_512_0            last=0.64286    mean=0.65377    std=0.02161    max=0.70244    min=0.60733   

mean of max in [30, 60] mean=0.700501948595047 std=0.004589613498755857
mean of all [30, 60] mean=0.6496806547045708 std=0.02587890539033434
mean of last in 60 mean=0.6463367789983749 std=0.020882646896088007


## Great Expansion on Dataset

In [19]:
plotter_vert(filtering_subfolders(["dl-healthver"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-healthver_seed-42       last=0.61404    mean=0.63678    std=0.02353    max=0.66923    min=0.57292   
bert_512_comb-dl-healthver_seed-1M       last=0.61033    mean=0.52244    std=0.03031    max=0.5641     min=0.4625    
bert_512_comb-dl-healthver_seed-1998     last=0.62222    mean=0.6018     std=0.01984    max=0.63025    min=0.54455   
bert_512_comb-dl-healthver_seed-0        last=0.63415    mean=0.59541    std=0.02859    max=0.64706    min=0.53333   

mean of max in [30, 49] mean=0.6276610791683197 std=0.03921322046799351
mean of all [30, 49] mean=0.5891066279850508 std=0.0489928099865604
mean of last in 49 mean=0.620183065533638 std=0.009138368751063468


In [20]:
plotter_vert(filtering_subfolders(["ds-healthver"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-healthver_seed-42       last=0.66071    mean=0.62112    std=0.02631    max=0.66667    min=0.56995   
bert_512_comb-ds-healthver_seed-1M       last=0.63071    mean=0.62158    std=0.04052    max=0.66667    min=0.46707   
bert_512_comb-ds-healthver_seed-1998     last=0.61261    mean=0.63099    std=0.01353    max=0.65574    min=0.60513   
bert_512_comb-ds-healthver_seed-0        last=0.63717    mean=0.63615    std=0.0211     max=0.67234    min=0.60317   

mean of max in [30, 49] mean=0.6653528809547424 std=0.006015190571494298
mean of all [30, 49] mean=0.6274601707333013 std=0.027950443608478308
mean of last in 49 mean=0.6353001147508621 std=0.01721339010809751


In [21]:
plotter_vert(filtering_subfolders(["dl-scifact"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-scifact_seed-42         last=0.54709    mean=0.56706    std=0.0228     max=0.61194    min=0.53394   
bert_512_comb-dl-scifact_seed-1M         last=0.56067    mean=0.50394    std=0.04863    max=0.58921    min=0.41808   
bert_512_comb-dl-scifact_seed-1998       last=0.60274    mean=0.60111    std=0.02065    max=0.63115    min=0.54795   
bert_512_comb-dl-scifact_seed-0          last=0.58577    mean=0.57875    std=0.02863    max=0.61088    min=0.50485   

mean of max in [30, 49] mean=0.6107945442199707 std=0.01484408058337341
mean of all [30, 49] mean=0.5627141971337167 std=0.0483092354354883
mean of last in 49 mean=0.5740671306848526 std=0.02160261363908109


In [22]:
plotter_vert(filtering_subfolders(["ds-scifact"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-scifact_seed-42         last=0.61947    mean=0.56978    std=0.04136    max=0.61468    min=0.45455   
bert_512_comb-ds-scifact_seed-1M         last=0.64486    mean=0.60481    std=0.03505    max=0.64889    min=0.52459   
bert_512_comb-ds-scifact_seed-1998       last=0.66355    mean=0.66697    std=0.01405    max=0.6888     min=0.64186   
bert_512_comb-ds-scifact_seed-0          last=0.6867     mean=0.66219    std=0.01886    max=0.70386    min=0.625     

mean of max in [30, 49] mean=0.6640567928552628 std=0.0348740963177913
mean of all [30, 49] mean=0.6259375265554378 std=0.050224723556899636
mean of last in 49 mean=0.6536438763141632 std=0.02467606688962441


In [23]:
plotter_vert(filtering_subfolders(["dl-snli"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-snli_seed-42            last=0.61333    mean=0.61779    std=0.01402    max=0.63673    min=0.59155   
bert_512_comb-dl-snli_seed-1M            last=0.57297    mean=0.58258    std=0.07661    max=0.66942    min=0.4       
bert_512_comb-dl-snli_seed-1998          last=0.68548    mean=0.66314    std=0.01252    max=0.68526    min=0.63208   
bert_512_comb-dl-snli_seed-0             last=0.59716    mean=0.62389    std=0.01511    max=0.65438    min=0.59701   

mean of max in [30, 49] mean=0.6614482551813126 std=0.017967158763058943
mean of all [30, 49] mean=0.6218504580227953 std=0.04929038590143548
mean of last in 49 mean=0.6172366440296173 std=0.04193867633656989


In [24]:
plotter_vert(filtering_subfolders(["ds-snli"]), [0, 44], [30, 44], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-snli_seed-42            last=0.66923    mean=0.61126    std=0.04444    max=0.66148    min=0.47674   
bert_512_comb-ds-snli_seed-1M            last=0.63393    mean=0.63606    std=0.02824    max=0.68333    min=0.56863   
bert_512_comb-ds-snli_seed-1998          last=0.64815    mean=0.63784    std=0.01913    max=0.67826    min=0.6087    
bert_512_comb-ds-snli_seed-0             last=0.61883    mean=0.63359    std=0.02655    max=0.66667    min=0.56522   

mean of max in [30, 44] mean=0.6724348664283752 std=0.008746871355765464
mean of all [30, 44] mean=0.6296886226960591 std=0.03280870154254667
mean of last in 44 mean=0.642535388469696 std=0.01857400419630752


In [25]:
plotter_vert(filtering_subfolders(["dl-scihv"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-scihv_seed-42           last=0.56364    mean=0.60413    std=0.02488    max=0.63934    min=0.55708   
bert_512_comb-dl-scihv_seed-1M           last=0.57014    mean=0.57848    std=0.02629    max=0.62097    min=0.52535   
bert_512_comb-dl-scihv_seed-1998         last=0.5514     mean=0.56926    std=0.02614    max=0.61135    min=0.49711   
bert_512_comb-dl-scihv_seed-0            last=0.57983    mean=0.52013    std=0.04241    max=0.57143    min=0.44565   

mean of max in [30, 49] mean=0.610773578286171 std=0.024842352202701572
mean of all [30, 49] mean=0.5680014106788134 std=0.043305404723205425
mean of last in 49 mean=0.5662514865398407 std=0.010330384171023594


In [26]:
plotter_vert(filtering_subfolders(["ds-scihv"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-scihv_seed-42           last=0.63717    mean=0.58073    std=0.0316     max=0.66383    min=0.54054   
bert_512_comb-ds-scihv_seed-1M           last=0.64889    mean=0.58663    std=0.03963    max=0.64979    min=0.5       
bert_512_comb-ds-scihv_seed-1998         last=0.657      mean=0.66154    std=0.02334    max=0.70222    min=0.6129    
bert_512_comb-ds-scihv_seed-0            last=0.66379    mean=0.62024    std=0.02683    max=0.65833    min=0.54271   

mean of max in [30, 49] mean=0.6685436069965363 std=0.020077677522722173
mean of all [30, 49] mean=0.6122853959861555 std=0.04466019428920544
mean of last in 49 mean=0.6517137438058853 std=0.009917909645064171


In [27]:
plotter_vert(filtering_subfolders(["dl-scisnli"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-scisnli_seed-42         last=0.60656    mean=0.59388    std=0.02817    max=0.64151    min=0.53535   
bert_512_comb-dl-scisnli_seed-1M         last=0.56652    mean=0.57325    std=0.01752    max=0.61719    min=0.54386   
bert_512_comb-dl-scisnli_seed-1998       last=0.54455    mean=0.59692    std=0.03682    max=0.65201    min=0.51042   
bert_512_comb-dl-scisnli_seed-0          last=0.64542    mean=0.5725     std=0.03228    max=0.6332     min=0.50761   

mean of max in [30, 49] mean=0.6359790563583374 std=0.01273328147099
mean of all [30, 49] mean=0.5841372530711325 std=0.031667156246413336
mean of last in 49 mean=0.5907634496688843 std=0.03859872068004604


In [28]:
plotter_vert(filtering_subfolders(["ds-scisnli"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-scisnli_seed-42         last=0.57436    mean=0.62933    std=0.02413    max=0.66667    min=0.59      
bert_512_comb-ds-scisnli_seed-1M         last=0.64348    mean=0.64838    std=0.02369    max=0.68548    min=0.58883   
bert_512_comb-ds-scisnli_seed-1998       last=0.62439    mean=0.66134    std=0.02332    max=0.69804    min=0.59686   
bert_512_comb-ds-scisnli_seed-0          last=0.65766    mean=0.65746    std=0.02158    max=0.69291    min=0.61836   

mean of max in [30, 49] mean=0.6857758015394211 std=0.011901478528056212
mean of all [30, 49] mean=0.6491255234730872 std=0.026289371303786807
mean of last in 49 mean=0.6249713003635406 std=0.03151527522663295


In [29]:
plotter_vert(filtering_subfolders(["dl-hvsnli"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-dl-hvsnli_seed-42          last=0.53684    mean=0.55028    std=0.02797    max=0.60204    min=0.49162   
bert_512_comb-dl-hvsnli_seed-1M          last=0.65801    mean=0.57785    std=0.04242    max=0.66071    min=0.51534   
bert_512_comb-dl-hvsnli_seed-1998        last=0.61538    mean=0.65249    std=0.01387    max=0.67939    min=0.62882   
bert_512_comb-dl-hvsnli_seed-0           last=0.61538    mean=0.55931    std=0.05229    max=0.63248    min=0.40237   

mean of max in [30, 49] mean=0.6436557620763779 std=0.02926007847533332
mean of all [30, 49] mean=0.5849807807489446 std=0.05472757076801012
mean of last in 49 mean=0.6064050048589706 std=0.04376984959167594


In [30]:
plotter_vert(filtering_subfolders(["ds-hvsnli"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert_512_comb-ds-hvsnli_seed-42          last=0.6114     mean=0.62795    std=0.03043    max=0.67568    min=0.55249   
bert_512_comb-ds-hvsnli_seed-1M          last=0.60177    mean=0.5857     std=0.02864    max=0.63964    min=0.51852   
bert_512_comb-ds-hvsnli_seed-1998        last=0.66376    mean=0.61139    std=0.02779    max=0.64865    min=0.55738   
bert_512_comb-ds-hvsnli_seed-0           last=0.66667    mean=0.64207    std=0.03246    max=0.69298    min=0.54945   

mean of max in [30, 49] mean=0.6642365902662277 std=0.021243661286901035
mean of all [30, 49] mean=0.6167758303253275 std=0.03650803791508
mean of last in 49 mean=0.6358977556228638 std=0.029528294892179938


## Model Test

In [32]:
plotter_vert(filtering_subfolders(["bert-v2-512_5"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v2-512_5.000e-6_seed-42             last=0.61905    mean=0.6429     std=0.01538    max=0.66667    min=0.61033   
bert-v2-512_5.000e-6_seed-1M             last=0.66379    mean=0.64506    std=0.02093    max=0.68376    min=0.6       
bert-v2-512_5.000e-6_seed-1998           last=0.60396    mean=0.62928    std=0.02053    max=0.6696     min=0.59184   
bert-v2-512_5.000e-6_seed-0              last=0.65158    mean=0.62574    std=0.01813    max=0.65812    min=0.58462   

mean of max in [30, 49] mean=0.6695376485586166 std=0.009231902156745136
mean of all [30, 49] mean=0.6357461206222835 std=0.020643671843771246
mean of last in 49 mean=0.6345962136983871 std=0.024090167321773274


In [33]:
plotter_vert(filtering_subfolders(["bert-v3-512_5"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v3-512_5.000e-6_seed-42             last=0.58291    mean=0.63996    std=0.013      max=0.65801    min=0.61468   
bert-v3-512_5.000e-6_seed-1M             last=0.61084    mean=0.63149    std=0.01821    max=0.66087    min=0.58291   
bert-v3-512_5.000e-6_seed-1998           last=0.57282    mean=0.58745    std=0.02391    max=0.62673    min=0.54023   
bert-v3-512_5.000e-6_seed-0              last=0.70833    mean=0.67991    std=0.02004    max=0.70175    min=0.63054   

mean of max in [30, 49] mean=0.661840170621872 std=0.026653442890660736
mean of all [30, 49] mean=0.6347026887692904 std=0.038042859467823806
mean of last in 49 mean=0.6187252253293991 std=0.05357687768966202


In [34]:
plotter_vert(filtering_subfolders(["bert-v4-512_5"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v4-512_5.000e-6_seed-42             last=0.63111    mean=0.62183    std=0.01922    max=0.64574    min=0.57143   
bert-v4-512_5.000e-6_seed-1M             last=0.63964    mean=0.6445     std=0.01709    max=0.66964    min=0.6087    
bert-v4-512_5.000e-6_seed-1998           last=0.61751    mean=0.61239    std=0.01921    max=0.66372    min=0.57436   
bert-v4-512_5.000e-6_seed-0              last=0.63793    mean=0.62049    std=0.02224    max=0.6722     min=0.58763   

mean of max in [30, 49] mean=0.6628246903419495 std=0.010332645071489854
mean of all [30, 49] mean=0.6248014514383516 std=0.02288461968286741
mean of last in 49 mean=0.6315483152866364 std=0.008709642418263748


In [35]:
plotter_vert(filtering_subfolders(["bert-v5-512_5"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v5-512_5.000e-6_seed-42             last=0.60287    mean=0.63185    std=0.01525    max=0.66368    min=0.60177   
bert-v5-512_5.000e-6_seed-1M             last=0.66397    mean=0.64671    std=0.01883    max=0.67433    min=0.60759   
bert-v5-512_5.000e-6_seed-1998           last=0.63717    mean=0.64053    std=0.01561    max=0.66397    min=0.60684   
bert-v5-512_5.000e-6_seed-0              last=0.61411    mean=0.62984    std=0.00999    max=0.64979    min=0.61044   

mean of max in [30, 49] mean=0.6629408299922943 std=0.008721645465624078
mean of all [30, 49] mean=0.6372337011914504 std=0.01669313021574043
mean of last in 49 mean=0.6295286267995834 std=0.023413902346044443


In [36]:
plotter_vert(filtering_subfolders(["bert-v6-512_5"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v6-512_5.000e-6_seed-42             last=0.675      mean=0.65364    std=0.01409    max=0.67249    min=0.61135   
bert-v6-512_5.000e-6_seed-1M             last=0.64574    mean=0.64336    std=0.01397    max=0.66403    min=0.61674   
bert-v6-512_5.000e-6_seed-1998           last=0.66667    mean=0.64933    std=0.01416    max=0.67433    min=0.61244   
bert-v6-512_5.000e-6_seed-0              last=0.66667    mean=0.65662    std=0.01415    max=0.67633    min=0.61836   

mean of max in [30, 49] mean=0.6717946827411652 std=0.004683161573508654
mean of all [30, 49] mean=0.6507393019764047 std=0.014950260965332363
mean of last in 49 mean=0.6635183244943619 std=0.010813480348291604


In [37]:
plotter_vert(filtering_subfolders(["bert-v7"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v7-512_5.000e-6_seed-42             last=0.63415    mean=0.57535    std=0.03662    max=0.64348    min=0.5       
bert-v7-512_5.000e-6_seed-1998           last=0.58407    mean=0.58949    std=0.01417    max=0.61111    min=0.55769   

mean of max in [30, 49] mean=0.6272946894168854 std=0.016183584928512573
mean of all [30, 49] mean=0.5824209294821087 std=0.028655130046773874
mean of last in 49 mean=0.609108567237854 std=0.025037765502929688


In [38]:
plotter_vert(filtering_subfolders(["bert-v8"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v8-512_5.000e-6_seed-42             last=0.60606    mean=0.62379    std=0.02667    max=0.65823    min=0.55914   
bert-v8-512_5.000e-6_seed-1998           last=0.62979    mean=0.59669    std=0.03112    max=0.65217    min=0.53476   

mean of max in [30, 49] mean=0.6552008986473083 std=0.0030269622802734375
mean of all [30, 49] mean=0.6102385536620492 std=0.03199440979030666
mean of last in 49 mean=0.6179239153862 std=0.011863291263580322


In [39]:
plotter_vert(filtering_subfolders(["bert-v9"]), [0, 49], [30, 49], dontplot=True)

------ valid_f1 ------
bert-v9-cl_v4                            last=0.59       mean=0.63709    std=0.01886    max=0.68103    min=0.59184   
bert-v9-cl_v3                            last=0.61404    mean=0.59641    std=0.02404    max=0.64317    min=0.52514   
bert-v9-cl_v2                            last=0.63111    mean=0.6188     std=0.02635    max=0.67241    min=0.56311   
bert-v9-cl_v1                            last=0.63755    mean=0.63542    std=0.02409    max=0.68085    min=0.59048   
bert-v9-512_5.000e-6_seed-42             last=0.625      mean=0.62654    std=0.02017    max=0.65217    min=0.57711   
bert-v9-512_5.000e-6_seed-1M             last=0.65789    mean=0.65743    std=0.01262    max=0.68595    min=0.63063   

mean of max in [30, 49] mean=0.6692659060160319 std=0.01598976279413276
mean of all [30, 49] mean=0.6286164726081648 std=0.02846215657063416
mean of last in 49 mean=0.6259325742721558 std=0.02088476179100778
